In [1]:
from module.midiExtractor import midiExtractor

file = midiExtractor("select-midi/1238.midi")
file.extract()
file.encodeNotes()
file.encodeDuration()
None

In [2]:
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU, Embedding, Concatenate, LSTM
from keras.models import Sequential, Model
import numpy as np
from keras.optimizers import Adam

LATENT_DIM = 20
DATA_SHAPE = (100,)
DATA_LEN = 100
OPTIMIZER = Adam(0.0002, 0.5)


def build_decoder():

    ix = Input(shape=(None,LATENT_DIM))

    x = Dense(32)(ix)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dense(64)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dense(128)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dense(512)(x)
    x = LeakyReLU(alpha=0.2)(x)
    notes = Dense(DATA_LEN, activation='softmax')(x)
    duration = Dense(DATA_LEN, activation='softmax')(x)
    
    return Model(ix, [notes, duration])

def build_encoder():

    notes = Input(shape=DATA_SHAPE)
    
    notes = Embedding(input_dim=100, output_dim=100)(notes)

    duration = Input(shape=DATA_SHAPE)
    duration = Embedding(input_dim=100, output_dim=100)(duration)

    x = Concatenate()([notes, duration])
    # x = LSTM(512)(x)
    x = Dense(128)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dense(64)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dense(32)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dense(LATENT_DIM)(x)

    return Model([notes, duration], x)

def create_model():
    encoder = build_encoder()
    decoder = build_decoder()

    nz = Input(shape=DATA_SHAPE)
    dz = Input(shape=DATA_SHAPE)
    encoded_repr = encoder([nz, dz])
    [reconstitued_notes, reconstitued_duration] = decoder(encoded_repr)
    
    combined = Model([nz, dz], [reconstitued_notes, reconstitued_duration])

    combined.compile(loss=['mse', 'mse'], optimizer=OPTIMIZER)

    return combined, encoder, decoder

class Autoencoder():
    def __init__(self) -> None:
        self.combined, self.encoder, self.decoder  = create_model()

    def train(self, epochs, batch_size=128, sample_interval=50):

        for epoch in range(epochs):

            # ---------------------
            #  Train Autoencoder
            # ---------------------

            # Select a random batch of images

            # Train the autoencoder

            Notes = np.array(file.notes[:100]).reshape(1,100)
            Duration = np.array(file.durations[:100]).reshape(1,100)

            g_loss = self.combined.train_on_batch([Notes, Duration], [Notes, Duration])
            # Plot the progress
            print ("%d [MSE loss: %f]" % (epoch, g_loss[0]))




2024-02-04 22:59:54.804611: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-04 22:59:54.829046: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-04 22:59:54.829076: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-04 22:59:54.829719: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-04 22:59:54.833982: I tensorflow/core/platform/cpu_feature_guar

In [3]:
ae = Autoencoder()
ae.train(epochs=1000, batch_size=32, sample_interval=200)

0 [MSE loss: 1748.630249]
1 [MSE loss: 1748.376831]
2 [MSE loss: 1748.269897]
3 [MSE loss: 1748.261841]
4 [MSE loss: 1748.260742]
5 [MSE loss: 1748.260376]
6 [MSE loss: 1748.260376]
7 [MSE loss: 1748.260376]
8 [MSE loss: 1748.260254]
9 [MSE loss: 1748.260254]
10 [MSE loss: 1748.260254]
11 [MSE loss: 1748.260254]
12 [MSE loss: 1748.260254]
13 [MSE loss: 1748.260254]
14 [MSE loss: 1748.260254]
15 [MSE loss: 1748.260010]
16 [MSE loss: 1748.260010]
17 [MSE loss: 1748.260132]
18 [MSE loss: 1748.260132]
19 [MSE loss: 1748.260132]
20 [MSE loss: 1748.260132]
21 [MSE loss: 1748.260132]
22 [MSE loss: 1748.260132]
23 [MSE loss: 1748.260132]
24 [MSE loss: 1748.260132]
25 [MSE loss: 1748.260132]
26 [MSE loss: 1748.260132]
27 [MSE loss: 1748.260132]
28 [MSE loss: 1748.260132]
29 [MSE loss: 1748.260132]
30 [MSE loss: 1748.260010]
31 [MSE loss: 1748.260132]
32 [MSE loss: 1748.260132]
33 [MSE loss: 1748.259888]
34 [MSE loss: 1748.259766]
35 [MSE loss: 1748.259521]
36 [MSE loss: 1748.258911]
37 [MSE los